In [ ]:
# 30/08/2022
# Author: Andrea Mendonça Inácio

# Fonte:
# https://archive.ics.uci.edu/ml/datasets/Bank+Marketing#

# Link download: DATASET - CSV: 
# https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip


# The data is related with direct marketing campaigns of a Portuguese banking institution. 
# The marketing campaigns were based on phone calls. Often, more than one contact to 
# the same client was required, in order to access if the product (bank term deposit) 
# would be ('yes') or not ('no') subscribed.

# The classification goal is to predict if the client will subscribe (yes/no) a term deposit (variable y).
#####


# Dicionario de dados
# bank client data:
#0 - age (numeric)
#1 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
#2 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
#3 - education (categorical: 'primary', 'secondary', 'tertiary')
#4 - default: has credit in default? (categorical: 'no','yes','unknown')
#5 - balance: ?
#6 - housing: has housing loan? (categorical: 'no','yes','unknown')
#7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
#8 - contact: contact communication type (categorical: 'cellular','telephone')
#9 - day: last contact day of the month
#10 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
#11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). 
#     Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. 
#     Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
#12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
#13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
#14 - previous: number of contacts performed before this campaign and for this client (numeric)
#15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
#Output variable (desired target):
#16 - y - has the client subscribed a term deposit? (binary: 'yes','no')


# Nova seção

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import sqlite3

In [ ]:
df = pd.read_csv ('bank.csv', delimiter=';')

In [ ]:
df.info()

In [ ]:
# Ver a soma de valores ausentes
df.isnull().sum()

In [ ]:
# Drop de linhas que contém qualquer valor nulo
df.dropna(axis=0, how='all', inplace=True)

In [ ]:
# Feature Engeneering (pegar somente 8 opçoes principais para variavel job)

# Usando apply com lambda para diminuir opcoes
df['job'] = df['job'].apply(lambda x: 'other' if x not in ['management','blue-collar','technician','admin.','services', 'retired', 'unemployed'] else x)

# Visualizando count da variavel job apos a alteracao
df.job.value_counts()


management     969
blue-collar    946
technician     768
other          585
admin.         478
services       417
retired        230
unemployed     128
Name: job, dtype: int64

In [ ]:
# Transformando a Variavel age para 4 intervalos em nova variavel Range_Age

df['Range_Age'] = pd.cut(df['age'].astype('int64'),bins=4)
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,Range_Age
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no,"(18.932, 36.0]"
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no,"(18.932, 36.0]"
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no,"(18.932, 36.0]"
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no,"(18.932, 36.0]"
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no,"(53.0, 70.0]"


In [ ]:
# Criar a feature Dia/Mes (unindo as variaveis)
df['day'] = df['day'].astype('str')
df['day_month'] = df['day']+('/')+df['month']

In [ ]:
# Excluir variaveis desnecessarias day,month,balance,pdays
df.drop(['day','month','balance','pdays'], axis=1, inplace=True)

In [ ]:
# Aplicar Label Enconder nas variaveis binarias: default,housing,loan,y

df['default'] = df.default.astype('category').cat.codes
df['housing'] = df.housing.astype('category').cat.codes
df['loan'] = df.loan.astype('category').cat.codes
df['y'] = df.y.astype('category').cat.codes


In [ ]:
# Aplicar One-Hot Enconder na variavel education
df['contact'].astype('str')
df = pd.get_dummies(df, columns=['contact'])

In [ ]:
# Visualizando 5 primeiras linhas
df.head()

,age,job,marital,education,default,housing,loan,duration,campaign,previous,poutcome,y,Range_Age,day_month,contact_cellular,contact_telephone,contact_unknown
0,30,unemployed,married,primary,0,0,0,79,1,0,unknown,0,"(18.932, 36.0]",19/oct,1,0,0
1,33,services,married,secondary,0,1,1,220,1,4,failure,0,"(18.932, 36.0]",11/may,1,0,0
2,35,management,single,tertiary,0,1,0,185,1,1,failure,0,"(18.932, 36.0]",16/apr,1,0,0
3,30,management,married,tertiary,0,1,1,199,4,0,unknown,0,"(18.932, 36.0]",3/jun,0,0,1
4,59,blue-collar,married,secondary,0,1,0,226,1,0,unknown,0,"(53.0, 70.0]",5/may,0,0,1


In [ ]:
df_aux = df.loc[df['poutcome'] == 'success']
df_aux.head()

,age,job,marital,education,default,housing,loan,duration,campaign,previous,poutcome,y,Range_Age,day_month,contact_cellular,contact_telephone,contact_unknown
49,61,admin.,married,unknown,0,1,0,181,1,1,success,1,"(53.0, 70.0]",27/jan,1,0,0
70,37,management,married,tertiary,0,0,0,268,2,3,success,1,"(36.0, 53.0]",16/jul,1,0,0
98,36,blue-collar,divorced,secondary,0,0,0,473,1,1,success,0,"(18.932, 36.0]",12/feb,1,0,0
110,21,other,single,secondary,0,0,0,258,6,3,success,1,"(18.932, 36.0]",30/jun,1,0,0
156,55,admin.,married,secondary,0,0,0,324,3,2,success,1,"(53.0, 70.0]",1/jun,1,0,0


In [ ]:
# Visualizando clientes que fecharam campanha de depósito (variavel 'y'=1), comparando por job e Range_Age

filtro = ['Range_Age', 'y','job']
df[filtro].value_counts().unstack('Range_Age')

Range_Age      (18.932, 36.0]  (36.0, 53.0]  (53.0, 70.0]  (70.0, 87.0]
y job                                                                  
0 admin.                190.0         186.0          44.0           NaN
  blue-collar           357.0         438.0          80.0           2.0
  management            351.0         380.0         106.0           1.0
  other                 186.0         248.0          74.0           2.0
  retired                 3.0          21.0         128.0          24.0
  services              180.0         165.0          34.0           NaN
  technician            314.0         324.0          46.0           1.0
  unemployed             44.0          57.0          14.0           NaN
1 admin.                 22.0          30.0           6.0           NaN
  blue-collar            36.0          30.0           3.0           NaN
  management             63.0          57.0          11.0           NaN
  other                  34.0          27.0          13.0           1.0
  retired                 NaN           2.0          29.0          23.0
  services               21.0          15.0           2.0           NaN
  technician             34.0          38.0          11.0           NaN
  unemployed              5.0           7.0           1.0           NaN

In [ ]:
df.describe(include=['object'])

,job,marital,education,poutcome,day_month
count,4521,4521,4521,4521,4521
unique,8,3,4,4,256
top,management,married,secondary,unknown,15/may
freq,969,2797,2306,3705,114


In [ ]:
# Mediana da idade dos clientes que fecharam negocio nesta campanha
df[df['y'] == 1].age.median()


40.0

In [ ]:
# Visualizando perfil pessoal de clientes que fecharam negocio nesta campanha (variavel y='1') por estado civil (variavel='marital') 
#  e nivel de escolaridade (variavel='education')

df[['marital','education', 'y']].groupby(['marital','education']).agg(['count'])


y
                   count
marital  education      
divorced primary      79
         secondary   270
         tertiary    155
         unknown      24
married  primary     526
         secondary  1427
         tertiary    727
         unknown     117
single   primary      73
         secondary   609
         tertiary    468
         unknown      46

In [ ]:
# Este grafico revela que o numero de contatos feito nesta campanha para um mesmo cliente está mais concentrado
# entre os 27 e 59 anos de idade, e para esta faixa o numero de tentativas se concentra entre 10 e 20.  
fig= px.scatter(df, x='age', y='campaign', color='campaign', size='campaign')
fig.show()

In [ ]:
# Este grafico revela que para esta campanha somente 11,5% dos clientes assinaram o termo de depósito
fig = px.pie (df, names='y', title='Clientes que assinaram o termo de depósito',)
fig.show()

In [ ]:
from plotly .subplots import make_subplots
fig = make_subplots(rows=4, cols=2, specs=[[ {"type": "pie"}, {"type": "pie"}],[ {"type": "pie"}, {"type": "pie"}],[ {"type": "pie"}, {"type": "pie"}],[ {"type": "pie"}, {"type": "pie"}]])

fig.add_trace(
go.Pie (labels=df.loc[df['job']=='management']['y'], pull=[.1,.1], title='Clientes assinaram contrato vs job Management'), row=1, col=1)
fig.add_trace(
go.Pie (labels=df.loc[df['job']=='blue-collar']['y'], pull=[.1,.1], title='Clientes assinaram contrato vs job Blue-Collar'), row=1, col=2)

fig.add_trace(
go.Pie (labels=df.loc[df['job']=='technician']['y'], pull=[.1,.1], title='Clientes assinaram contrato vs job Technician'), row=2, col=1)
fig.add_trace(
go.Pie (labels=df.loc[df['job']=='other']['y'], pull=[.1,.1], title='Clientes assinaram contrato vs job Other'), row=2, col=2)

fig.add_trace(
go.Pie (labels=df.loc[df['job']=='admin.']['y'], pull=[.1,.1], title='Clientes assinaram contrato vs job Admin'), row=3, col=1)
fig.add_trace(
go.Pie (labels=df.loc[df['job']=='services']['y'], pull=[.1,.1], title='Clientes assinaram contrato vs job Services'), row=3, col=2)

fig.add_trace(
go.Pie (labels=df.loc[df['job']=='retired']['y'], pull=[.1,.1], title='Clientes assinaram contrato vs job Retired'), row=4, col=1)
fig.add_trace(
go.Pie (labels=df.loc[df['job']=='unemployed']['y'], pull=[.1,.1], title='Clientes assinaram contrato vs job Unemployed'), row=4, col=2)

In [ ]:
# Histograma com boxplot
# Neste histograma vemos que a faixa entre 30 a 34 anos de idade foi a que representou a maior frequência nesta campanha

fig = px.histogram(df, x='age', nbins=30, marginal='box')
fig.show()